In [72]:
import pandas as pd
import requests
import sys
import os
import time

# Add the parent directory of 'utils' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.classes.Settings import Settings
from utils.shared import get_duration_message

In [ ]:
EXTERNAL_API_HEADERS = {
    "Authorization": f"Bearer {Settings().BEARER_TOKEN}"
}
EXTERNAL_API_NAMESPACE='prodavnicaalata'

def get_products_from_external_api():
    now = pd.Timestamp.now()

    base_url = (
        f"{Settings().API_URL}/GET/products/"
        f"?namespace={EXTERNAL_API_NAMESPACE}"
    )

    limit_from = 0
    limit_range = 1000

    all_products = []  # will hold dicts from each batch

    while True:
        print(f"Fetching raw data from: {limit_from} to: {limit_from + limit_range}\n")

        url = (
            f"{base_url}"
            f"&limit={limit_from},{limit_range}"
            f"&hide_categories=true&hide_seo=true&hide_tags=true&hide_manufacturer=true&hide_items=true&hide_attributes=true&hide_locations=true&hide_variations=true"
        )

        response = requests.get(url, headers=EXTERNAL_API_HEADERS)

        if response.status_code == 200:
            json_payload = response.json()
            data_section = json_payload.get("data", {})
            batch_products = data_section.get("products", []) if data_section else []
            
            if batch_products == []:
                break

            all_products.extend(batch_products)
        else:
            raise Exception(f"External CB request failed: {response}.\n")

        limit_from = limit_from + limit_range
        
        time.sleep(0.05) # Tiny delay to avoid hammering the API

    products = pd.DataFrame(all_products, columns=[
        'title', 
        'description', 
        'text', 
        'category',
        'status',
    ])

    print(get_duration_message(now))

    return products

In [74]:
# products = get_products_from_external_api()
products = pd.read_csv('../../data/products.csv')

In [75]:
# products = products.loc[products['status'] == 'Published'].reset_index(drop=True)
# products.drop(columns='status', inplace=True)
# products.to_csv('../../data/products.csv', index=False)

In [76]:
products.head(10)

,title,description,text,category
0,"Bosch GWS 7-115 ugaona brusilica, 720W","Bosch brusilica GWS 7-115 , 720W - Najmanji ob...","<h2><strong>Bosch GWS 7-115 ugaona brusilica ,...","[[1],[1000485],[8]]"
1,Bosch GSR 10.8-2-LI Akumulatorska šrafilica,"Aku.šrafilica - GSR 10.8-2-LI,Bosch.Najkraći 1...","<h2>Aku.&scaron;rafilica - GSR 10.8-2-LI,Bosch...","[[1000106],[4]]"
2,Wolfcraft 4688000 Zavijač pod uglom,"Ugao samo 52 mm,uključujući 25 mm bit nastavak...","<h2><strong><span style=""font-size: 1.75rem;"">...","[[1000285],[348],[87]]"
3,"Haupa Alat za izolovane papučice 0,5-6 210802","Alat za IZOLOVANE PAPUČICE 1,5-6 mm2 210802. \...","<div class=""Haupa-2022""><h2><strong>Haupa Alat...","[[2],[33],[101]]"
4,"Bosch GSB 13 RE vibraciona bušilica, 600W","Nominalna ulazna snaga 600W\r\nTežina 1,8kg\r\...","<h2><span style=""font-size: x-large;""><strong>...","[[1],[1000484],[5]]"
5,JSP zaštitne bistre naočare A-STILL,Jedinica mere: komad. \nEN standard: EN 166 1.F.,"<div class=""albo-2022""><h2><strong>JSP zaštitn...","[[1000429],[183],[180]]"
6,Bosch GBH 2400 Bušilica za beton SDS+,"Bušilica za beton SDS+ &amp;nbsp;- GBH 2400,Bo...","<h2>Bušilica za beton SDS+ &nbsp;- GBH 2400,Bo...","[[1],[1000484],[7]]"
7,"Bosch GWS 20-230 H ugaona brusilica, 2.000W",Profesionalna Bosch GWS 20-230 H ugaona brusil...,"<div><span style=""font-family: arial;""><strong...","[[1],[1000485],[8]]"
8,"Bosch GWS 1000 ugaona brusilica, 1.000 W","Bosch GWS 1000 ugaona brusilica , 1.000 WGWS 1...","<div><strong><span style=""font-size: x-large;""...","[[1],[1000485],[8]]"
9,"Bosch GWS 1400 Ugaona brusilica, 1.400W, 125 mm",Nominalna ulazna snaga* 1.400 W. \nBroj obrtaj...,"<div><strong><span style=""font-size: x-large;""...","[[1],[1000485],[8]]"


In [77]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68879 entries, 0 to 68878
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        68879 non-null  object
 1   description  68844 non-null  object
 2   text         68780 non-null  object
 3   category     67729 non-null  object
dtypes: object(4)
memory usage: 2.1+ MB


In [ ]:
model = 1

for grouped_products in products.groupby(['category']):
    print(len(grouped_products))
    # for product in grouped_products:
        # model(f'Based on the sent products, recommend the top 18 compatible products to: {grouped_products}. Products from the same category: {grouped_products}')

AttributeError: 'tuple' object has no attribute 'max'